In [ ]:
from os import walk
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
from ase.units import _e, _hplanck, kB

G0 = 2. * _e**2 / _hplanck

In [ ]:
def fermidistribution(energy, kt):
    # fermi level is fixed to zero
    # energy can be a single number or a list
    assert kt >= 0., 'Negative temperature encountered!'

    if kt == 0:
        if isinstance(energy, float):
            return int(energy / 2. <= 0)
        else:
            return (energy / 2. <= 0).astype(int)
    else:
        return 1. / (1. + np.exp(energy / kt))


def current(bias, energies, transmission, temp=300, unit='uA'):
    """Get the current in nA."""
    if not isinstance(bias, (int, float)):
        bias = bias[np.newaxis]
        energies = energies[:, np.newaxis]
        transmission = transmission[:, np.newaxis]

    fl = fermidistribution(energies - bias / 2., kB * temp)
    fr = fermidistribution(energies + bias / 2., kB * temp)

    return G0 * np.trapz((fl - fr) * transmission, x=energies, axis=0) * 1e6  # uA


In [ ]:
energies = np.load('../scatt/data_ENERGIES.npy').real

files = []
directory = 'transmission_data/'
for (dirpath, dirnames, filenames) in walk(directory):
    files.extend(f'{directory}{file}' for file in filenames)
files.sort(key=lambda fn: float(fn.split('_')[-1].strip(".npy")))

dV = 0.1
Vmin = -1.5
Vmax = 1.5 + dV / 2.
dmu_min = -4
dmu_max = 3
bias = np.arange(-2.5, 2.5 + dV / 2., dV)
temp = np.asarray([np.load(fn) for fn in files])
I = np.asarray([current(bias, energies, t, temp=9) for t in transmission])
# I = np.clip(I, 0, 30)
